In [2]:

import openai
import pandas as pd

In [3]:
OPENAI_API_KEY=""


In [5]:
openai.api_key = OPENAI_API_KEY
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

In [6]:
#this is the path for the readme_exp data
df_v1_PATH = "final_data.csv"
# df_v3_PATH = "df_v3_withcontext_remove_dup.csv"

df_v1_full = pd.read_csv(df_v1_PATH)
# df_v3 = pd.read_csv(df_v3_PATH)

In [7]:
print(len(df_v1_full))
df_v1_full.head()

308242


,Unnamed: 0.1,index,Unnamed: 0,id,text_to_annotate,start,end,ann_text,definition,split_print,UMLS+others
0,0,0,0,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3.0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...","['Virt', ' - ', 'Vite']"
1,1,2,2,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9.0,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,"['MG', ' ']"
2,2,3,3,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10.0,Oral,Taken by mouth.,['A substance intended for administration thro...,['Oral']
3,3,4,4,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11.0,Tablet,A pill.,['A tablet intended for oral administration.'],['Tablet']
4,4,5,5,98791999,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,11.0,14.0,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w...",['Vitamin B complex']


In [8]:

df_v1 = df_v1_full.drop(columns=['Unnamed: 0.1', 'index', 'Unnamed: 0', 'id'])


In [9]:
df_v1.head()

,text_to_annotate,start,end,ann_text,definition,split_print,UMLS+others
0,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,0.0,3.0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi...","['Virt', ' - ', 'The determination of the amou...","['Virt', ' - ', 'Vite']"
1,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,8.0,9.0,MG,"A tiny amount of something, usually a drug.",['A measure of weight. A milligram is approxim...,"['MG', ' ']"
2,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,9.0,10.0,Oral,Taken by mouth.,['A substance intended for administration thro...,['Oral']
3,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,10.0,11.0,Tablet,A pill.,['A tablet intended for oral administration.'],['Tablet']
4,Virt - Vite 2.5 - 25 - 1 MG Oral Tablet Vitami...,11.0,14.0,Vitamin B complex,A group of vitamins and nutrients. The body n...,"['A group of water-soluble vitamins, some of w...",['Vitamin B complex']


In [10]:
# prompt: unique pairs of column values in columns ann_text and definition in dataframe df_v1

unique_pairs = df_v1[['ann_text', 'definition']].drop_duplicates()
unique_terms = df_v1[['ann_text']].drop_duplicates()
unique_def = df_v1[['definition']].drop_duplicates()
print(len(unique_pairs))
print(len(unique_terms))
print(len(unique_def))


51623
42751
21317


In [11]:
unique_pairs.head(10)

,ann_text,definition
0,Virt - Vite,"A mix of vitamins. It provides vitamin B-6, vi..."
1,MG,"A tiny amount of something, usually a drug."
2,Oral,Taken by mouth.
3,Tablet,A pill.
4,Vitamin B complex,A group of vitamins and nutrients. The body n...
5,folic acid,A B vitamin.
6,vitamin c,A nutrient needed by the body to form and main...
7,tab,A pill.
8,po,By mouth.
9,qd,Every day.


In [12]:
import random

ind = random.randint(1, 200)

test_def = df_v1['definition'][ind]
test_ann = df_v1['ann_text'][ind]
print(test_def)
print(test_ann)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Generate a lay definition for the term that is similar in style and length to the definition"},
    {"role": "user", "content": "only output the lay definition of the term presented below in one small sentence without the word \"lay definition\""},
    {"role": "user", "content": f"term : {test_ann}, definition : {test_def}"}
  ]
)

print(completion.choices[0].message.content)



The area on a chromosome that codes for a particular inherited trait. Many diseases are caused by defective genes.
gene
A gene is a section of DNA that determines specific characteristics or traits passed down from parents.


In [15]:
# prompt: create a column "gpt_generated" and generate definitions similar to the "definition" in df_v1 using gpt3.5 of the openai

import openai

def generate_definition(row):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "Generate a lay definition for the term that is similar in style and length to the definition"},
      {"role": "user", "content": "only output the lay definition of the term presented below in one small sentence without the word \"lay definition\""},
      {"role": "user", "content": f"term : {row['ann_text']}, definition : {row['definition']}"}
    ]
  )

  return completion.choices[0].message.content

unique_pairs['gpt_generated'] = unique_pairs.apply(generate_definition, axis=1)



In [16]:
unique_pairs.to_csv("unique_dummy_v1.csv")